In [87]:
import numpy as np
import joblib
data_train = joblib.load('train.joblib')
data_test = joblib.load('eval1.joblib')

In [136]:
#Train
train_images = data_train['data'].reshape(-1, 2, 62, 47)
X_train = concatenated_images = np.concatenate([train_images[:, 0, :, :], train_images[:, 1, :, :]], axis=2)
y_train = data_train['target']
#Test
test_images = data_test['data'].reshape(-1, 2, 62, 47)
X_test = concatenated_images = np.concatenate([test_images[:, 0, :, :], test_images[:, 1, :, :]], axis=2)
y_test = data_test['target']

In [118]:
X_train.shape

(2200, 62, 94)

In [91]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

In [161]:
#augment as a pair image
from PIL import Image, ImageEnhance, ImageFilter
def image_augmentation(image):
    image = Image.fromarray(image)
    if np.random.rand() < 0.5:
        image = image.transpose(Image.FLIP_LEFT_RIGHT)
    if np.random.rand() < 0.5:
        angle = np.random.randint(-10, 10)
        image = image.rotate(angle)
    if np.random.rand() < 0.5:
        enhancer = ImageEnhance.Brightness(image)
        image = enhancer.enhance(np.random.uniform(0.9, 1.1))
    if np.random.rand() < 0.5:
        enhancer = ImageEnhance.Contrast(image)
        image = enhancer.enhance(np.random.uniform(0.9, 1.1))
    if np.random.rand() < 0.5:
        image = image.filter(ImageFilter.EDGE_ENHANCE)
    if np.random.rand() < 0.5:
        enhancer = ImageEnhance.Sharpness(image)
        image = enhancer.enhance(np.random.uniform(0.5, 2))

    return np.array(image)

In [143]:
#Augment as a pair
def augment_imagepairs(images, targets, dataset_multiplication):
    images = (images * 255).astype('uint8')
    images_augmented = []
    targets_augmented = []
    no_of_images = images.shape[0]

    for _ in range(dataset_multiplication):
        for i in range(no_of_images):
            augmented_image = augment_image(images[i])
            images_augmented.append(augmented_image)
            targets_augmented.append(targets[i])

    images_augmented = np.array(images_augmented)
    targets_augmented = np.array(targets_augmented)
    # Concatenate the augmented data with the original data
    images_combined = np.concatenate((images, images_augmented), axis=0)
    targets_combined = np.concatenate((targets, targets_augmented), axis=0)

    return images_combined, targets_combined


In [162]:
X_train_augmented_combined, y_train_augmented_combined = augment_imagepairs(X_train, y_train, 1)

In [163]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X_train_augmented_combined_flatten = X_train_augmented_combined.reshape(X_train_augmented_combined.shape[0], -1)
X_test_flatten = X_test.reshape(X_test.shape[0], -1)
X_train_flatten = X_train.reshape(X_train.shape[0], -1)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_augmented_combined_flatten)
X_test_scaled = scaler.fit_transform(X_test_flatten)
X_train_original_scale = scaler.fit_transform(X_train_flatten)

pca = PCA(n_components=150)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.fit_transform(X_test_scaled)
X_train_original_pca = pca.fit_transform(X_train_original_scale)


In [165]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

param_grid_svm = {
    'C':[10,100, 1000]
}
svm = SVC(probability=True, kernel='rbf', random_state=1)

grid_search_svm = GridSearchCV(estimator=svm, param_grid=param_grid_svm, cv=5, verbose=3, n_jobs=-1)
grid_search_svm.fit(X_train_pca, y_train_augmented_combined)

print("Best Parameters:", grid_search_svm.best_params_)
print("Best Cross-Validation Accuracy:", grid_search_svm.best_score_)

best_model_svm = grid_search_svm.best_estimator_
y_train_pred_svm = best_model_svm.predict(X_train_pca)
train_accuracy_svm = accuracy_score(y_train_augmented_combined, y_train_pred_svm)
print("Train Set Accuracy with Best Parameters:", train_accuracy_svm)
y_test_pred_svm = best_model_svm.predict(X_test_pca)
test_accuracy_svm = accuracy_score(y_test, y_test_pred_svm)
print("Test Set Accuracy with Best Parameters:", test_accuracy_svm)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
[CV 3/5] END .............................C=100;, score=0.764 total time=   9.6s
[CV 4/5] END ..............................C=10;, score=0.744 total time=   9.7s
[CV 4/5] END .............................C=100;, score=0.752 total time=  10.0s
[CV 2/5] END .............................C=100;, score=0.795 total time=  10.1s
[CV 5/5] END ..............................C=10;, score=0.742 total time=  10.2s
[CV 1/5] END ..............................C=10;, score=0.753 total time=  10.3s
[CV 5/5] END .............................C=100;, score=0.732 total time=  10.2s
[CV 2/5] END ..............................C=10;, score=0.794 total time=  10.3s
[CV 1/5] END .............................C=100;, score=0.752 total time=  10.4s
[CV 1/5] END ............................C=1000;, score=0.752 total time=  10.4s
[CV 3/5] END ..............................C=10;, score=0.760 total time=  10.6s
[CV 3/5] END ............................C=1000;,

In [92]:
#Augment separeately
def augment_dataset(images, targets, dataset_rounds):
    images_augmented = []
    targets_augmented = []

    no_of_images = images.shape[0]

    for i in range(dataset_rounds):  # Use `iteration` instead of `_`
        for j in range(no_of_images):  # Iterate over indices
            left_aug = augment_image(images[j, 0])  # Augment first image
            right_aug = augment_image(images[j, 1])  # Augment second image
            left_right_paired = np.stack([left_aug, right_aug], axis=0)
            images_augmented.append(left_right_paired)
            targets_augmented.append(targets[j])  # Access label using index

    images_augmented = np.array(images_augmented)
    targets_augmented = np.array(targets_augmented)

    return images_augmented, targets_augmented

In [102]:
images_train.dtype 

dtype('float32')

In [ ]:
def preprocess_images_pca_after_concat(images, n_components=150):
    no_of_images = images.shape[0]
    concatenated_images = images.reshape(no_of_images, -1)  # Shape: (n_samples, 2 * n_features)
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(concatenated_features)
    pca = PCA(n_components=n_components)
    reduced_features = pca.fit_transform(scaled_features)

    return reduced_features

In [ ]:
# Augment the training dataset
augmented_images_train, augmented_labels_train = augment_dataset(images_train, y_train, augment_times=2)

# Combine original and augmented datasets
images_train_combined = np.concatenate((images_train, augmented_images_train), axis=0)
labels_train_combined = np.concatenate((y_train, augmented_labels_train), axis=0)

# Preprocess the combined training dataset and test dataset with PCA after concatenation
X_train_processed = preprocess_images_pca_after_concat(images_train_combined, n_components=50)
X_test_processed = preprocess_images_pca_after_concat(images_test, n_components=50)

# Train the SVM model
clf = SVC(kernel='rbf', probability=True, random_state=1)
clf.fit(X_train_processed, labels_train_combined)

# Evaluate the model
y_train_pred = clf.predict(X_train_processed)
y_test_pred = clf.predict(X_test_processed)

print(f"Train Accuracy: {accuracy_score(labels_train_combined, y_train_pred) * 100:.2f}%")
print(f"Test Accuracy: {accuracy_score(y_test, y_test_pred) * 100:.2f}%")

# Visualize original and augmented images
visualize_augmentation(images_train[:5], augmented_images_train[:5])

In [8]:
from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()
X_train_scaled = std_scaler.fit_transform(X_train)
X_test_scaled = std_scaler.transform(X_test)


from sklearn.decomposition import PCA
pca = PCA(n_components=50,)
pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)
X_train_pca = pca.transform(X_train_scaled)


In [30]:
pca = PCA(n_components=150, whiten=True, svd_solver="randomized")
pca.fit_transform(X_train)
X_test_pca_w = pca.transform(X_test)
X_train_pca_w = pca.transform(X_train)

In [39]:
from sklearn.svm import SVC
clf = SVC()
clf.fit(X_train_pca_w, y_train)
y_pred_w = clf.predict(X_test_pca_w)
train_accuracy_svm_w = clf.score(X_train_pca_w, y_train)
test_accuracy_svm_w = clf.score(X_test_pca_w, y_test)
print(f"Train Set Accuracy SVM: {train_accuracy_svm_w * 100:.2f}%")
print(f"Test Set Accuracy SVM: {test_accuracy_svm_w * 100:.2f}%")

Train Set Accuracy SVM: 93.50%
Test Set Accuracy SVM: 53.30%


In [172]:
data_train['data'].shape

(2200, 5828)

In [182]:
#SVM pipeline polynomial
#Train
X_train = data_train['data']
y_train = data_train['target']
#Test
X_test = data_test['data']
y_test = data_test['target']

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the pipeline
pipeline_svm = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA(whiten=True, svd_solver="randomized")),                   
    ('svm', SVC(random_state=1))
])

# Define the parameter grid
param_grid_svm = {
    'pca__n_components': [200],
    'svm__C': [10000,1000],
    'svm__kernel': ['rbf'],
    'svm__gamma': [0.0001,0.01]
    # 'svm__degree': [2]
}

# Perform Grid Search
grid_search_svm = GridSearchCV(estimator=pipeline_svm, param_grid=param_grid_svm, cv=5, verbose=3, n_jobs=-1)
grid_search_svm.fit(X_train, y_train)

# Best parameters and score
print("Best Parameters:", grid_search_svm.best_params_)
print("Best Cross-Validation Accuracy:", grid_search_svm.best_score_)

# Evaluate on the test set
best_model_svm = grid_search_svm.best_estimator_
y_train_pred_svm = best_model_svm.predict(X_train)
train_accuracy_svm = accuracy_score(y_train, y_train_pred_svm)
print(f"Train Set Accuracy SVM: {train_accuracy_svm * 100:.2f}%")
y_test_pred_svm = best_model_svm.predict(X_test)
test_accuracy_svm = accuracy_score(y_test, y_test_pred_svm)
print(f"Test Set Accuracy SVM: {test_accuracy_svm * 100:.2f}%")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV 3/5] END pca__n_components=200, svm__C=10000, svm__gamma=0.01, svm__kernel=rbf;, score=0.559 total time=   1.7s
[CV 2/5] END pca__n_components=200, svm__C=10000, svm__gamma=0.01, svm__kernel=rbf;, score=0.525 total time=   1.7s
[CV 1/5] END pca__n_components=200, svm__C=10000, svm__gamma=0.01, svm__kernel=rbf;, score=0.589 total time=   1.8s
[CV 4/5] END pca__n_components=200, svm__C=10000, svm__gamma=0.01, svm__kernel=rbf;, score=0.550 total time=   1.9s
[CV 5/5] END pca__n_components=200, svm__C=10000, svm__gamma=0.01, svm__kernel=rbf;, score=0.552 total time=   1.9s
[CV 1/5] END pca__n_components=200, svm__C=1000, svm__gamma=0.0001, svm__kernel=rbf;, score=0.555 total time=   2.2s
[CV 5/5] END pca__n_components=200, svm__C=10000, svm__gamma=0.0001, svm__kernel=rbf;, score=0.582 total time=   2.9s
[CV 2/5] END pca__n_components=200, svm__C=10000, svm__gamma=0.0001, svm__kernel=rbf;, score=0.543 total time=   3.0s
[CV 4/5

In [18]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the pipeline
pipeline_rf = Pipeline([
    ('scaler', StandardScaler()),
    ('pca', PCA()),
    ('rf', RandomForestClassifier(random_state=1))
])

# Define the parameter grid for Grid Search
param_grid_rf = {
    'pca__n_components': [20, 50, 100, 500, 1000],
    'rf__n_estimators': [50, 100, 200, 500],        
    'rf__max_depth': [10, 20, 30],
    'rf__min_samples_split': [2, 5, 10],
    'rf__min_samples_leaf': [1, 2, 4],
    
}
# Perform Grid Search with Cross-Validation
grid_search_rf = GridSearchCV(estimator=pipeline_rf, param_grid=param_grid_rf, cv=5, verbose=2, n_jobs=-1)
grid_search_rf.fit(X_train, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search_rf.best_params_)
print("Best Cross-Validation Accuracy:", grid_search_rf.best_score_)

# Evaluate on the test set
best_model_rf = grid_search_rf.best_estimator_
y_train_pred_rf = best_model_rf.predict(X_train)
train_accuracy_rf = accuracy_score(y_train, y_train_pred_rf)
print(f"Train Set Accuracy RF: {train_accuracy_rf * 100:.2f}%")
y_test_pred_rf = best_model_rf.predict(X_test)
test_accuracy_rf = accuracy_score(y_test, y_test_pred_rf)
print(f"Test Set Accuracy RF: {test_accuracy_rf * 100:.2f}%")

Fitting 5 folds for each of 540 candidates, totalling 2700 fits
[CV] END pca__n_components=20, rf__max_depth=10, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50; total time=   0.8s
[CV] END pca__n_components=20, rf__max_depth=10, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50; total time=   0.8s
[CV] END pca__n_components=20, rf__max_depth=10, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50; total time=   0.8s
[CV] END pca__n_components=20, rf__max_depth=10, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50; total time=   0.8s
[CV] END pca__n_components=20, rf__max_depth=10, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=50; total time=   0.8s
[CV] END pca__n_components=20, rf__max_depth=10, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_estimators=100; total time=   1.0s
[CV] END pca__n_components=20, rf__max_depth=10, rf__min_samples_leaf=1, rf__min_samples_split=2, rf__n_est

/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/numpy/ma/core.py:2881: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Best Parameters: {'pca__n_components': 50, 'rf__max_depth': 30, 'rf__min_samples_leaf': 2, 'rf__min_samples_split': 5, 'rf__n_estimators': 500}
Best Cross-Validation Accuracy: 0.6468181818181818
Train Set Accuracy RF: 100.00%
Test Set Accuracy RF: 62.40%


In [19]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import matplotlib.pyplot as plt

# Define the pipeline
pipeline_lr = Pipeline([
    ('scaler', StandardScaler()),                  # Step 1: Scale features to zero mean and unit variance
    ('pca', PCA()),                                # Step 2: Dimensionality reduction with PCA
    ('logreg', LogisticRegression(random_state=1)) # Step 3: Logistic Regression
])

# Define the parameter grid for Grid Search
param_grid_lr = {
    'pca__n_components': [20, 50, 100, 500, 1000],            # Number of PCA components
    'logreg__C': [0.1, 1, 2, 5, 10],          # Regularization strength (inverse of lambda)
    'logreg__penalty': ['l1', 'l2'],               # Penalty type: L1 or L2 regularization
    'logreg__solver': ['liblinear', 'saga', 'lbfgs']        # Solver compatible with penalties
}

# Perform Grid Search with Cross-Validation
grid_search_lr = GridSearchCV(estimator=pipeline_lr, param_grid=param_grid_lr, cv=5, verbose=2, n_jobs=-1)
grid_search_lr.fit(X_train, y_train)

# Print the best parameters and cross-validation score
print("Best Parameters:", grid_search_lr.best_params_)
print(f"Best Cross-Validation Accuracy: {grid_search_lr.best_score_ * 100:.2f}%")

# Evaluate on the training set
best_model_lr = grid_search_lr.best_estimator_
y_train_pred_lr = best_model_lr.predict(X_train)
train_accuracy_lr = accuracy_score(y_train, y_train_pred_lr)
print(f"Train Set Accuracy Logistic Regression: {train_accuracy_lr * 100:.2f}%")

# Evaluate on the test set
y_test_pred_lr = best_model_lr.predict(X_test)
test_accuracy_lr = accuracy_score(y_test, y_test_pred_lr)
print(f"Test Set Accuracy Logistic Regression: {test_accuracy_lr * 100:.2f}%")

Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=liblinear, pca__n_components=20; total time=   0.5s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=liblinear, pca__n_components=20; total time=   0.5s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=liblinear, pca__n_components=20; total time=   0.5s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=liblinear, pca__n_components=20; total time=   0.6s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=liblinear, pca__n_components=20; total time=   0.6s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=liblinear, pca__n_components=50; total time=   0.7s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=liblinear, pca__n_components=50; total time=   0.7s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=liblinear, pca__n_components=50; total time=   0.7s
[CV] END logreg__C=0.1, logreg__penalty=l1, logre

/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=saga, pca__n_components=500; total time=   3.8s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=lbfgs, pca__n_components=20; total time=   0.4s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=lbfgs, pca__n_components=20; total time=   0.5s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=saga, pca__n_components=500; total time=   3.8s


/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=saga, pca__n_components=500; total time=   3.8s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=saga, pca__n_components=500; total time=   3.9s
[CV] END logreg__C=0.1, logreg__penalty=l1, logreg__solver=lbfgs, pca__n_components=20; total time=   0.5s


...

/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html

[CV] END logreg__C=10, logreg__penalty=l2, logreg__solver=lbfgs, pca__n_components=1000; total time=   4.2s
[CV] END logreg__C=10, logreg__penalty=l2, logreg__solver=lbfgs, pca__n_components=1000; total time=   4.0s
[CV] END logreg__C=10, logreg__penalty=l2, logreg__solver=lbfgs, pca__n_components=1000; total time=   3.9s


/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
125 fits failed out of a total of 750.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
125 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklea

Best Parameters: {'logreg__C': 0.1, 'logreg__penalty': 'l2', 'logreg__solver': 'saga', 'pca__n_components': 500}
Best Cross-Validation Accuracy: 54.73%
Train Set Accuracy Logistic Regression: 73.14%
Test Set Accuracy Logistic Regression: 52.50%


/opt/miniconda3/envs/com6018/lib/python3.12/site-packages/sklearn/linear_model/_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [9]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the parameter grid
param_grid_svm = {
    'C': [2],
    'degree': [2],
    

}

# Create the SVM model
svm = SVC(probability=True, kernel='poly', random_state=1)

# Perform Grid Search
grid_search_svm = GridSearchCV(estimator=svm, param_grid=param_grid_svm, cv=5, verbose=2, n_jobs=-1)
grid_search_svm.fit(X_train_pca, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search_svm.best_params_)
print("Best Cross-Validation Accuracy:", grid_search_svm.best_score_)

# Evaluate the best model on the test set
best_model_svm = grid_search_svm.best_estimator_
y_pred_svm = best_model_svm.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred_svm)
print("Test Set Accuracy with Best Parameters:", accuracy)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ......................................C=2, degree=2; total time=   0.4s
[CV] END ......................................C=2, degree=2; total time=   0.4s
[CV] END ......................................C=2, degree=2; total time=   0.4s
[CV] END ......................................C=2, degree=2; total time=   0.4s
[CV] END ......................................C=2, degree=2; total time=   0.4s
Best Parameters: {'C': 2, 'degree': 2}
Best Cross-Validation Accuracy: 0.6431818181818182
Test Set Accuracy with Best Parameters: 0.651


In [34]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the parameter grid
param_grid_svm = {
    'C': [1.5],

}

# Create the SVM model
svm = SVC(probability=True, kernel='rbf', random_state=1)

# Perform Grid Search
grid_search_svm = GridSearchCV(estimator=svm, param_grid=param_grid_svm, cv=5, verbose=2, n_jobs=-1)
grid_search_svm.fit(X_train_pca, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search_svm.best_params_)
print("Best Cross-Validation Accuracy:", grid_search_svm.best_score_)

# Evaluate the best model on the test set
best_model_svm = grid_search_svm.best_estimator_
y_pred_svm = best_model_svm.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred_svm)
print("Test Set Accuracy with Best Parameters:", accuracy)

Fitting 5 folds for each of 1 candidates, totalling 5 fits
[CV] END ..............................................C=1.5; total time=  15.7s
[CV] END ..............................................C=1.5; total time=  15.8s
[CV] END ..............................................C=1.5; total time=  15.8s
[CV] END ..............................................C=1.5; total time=  15.8s
[CV] END ..............................................C=1.5; total time=  15.8s
Best Parameters: {'C': 1.5}
Best Cross-Validation Accuracy: 0.6331818181818182
Test Set Accuracy with Best Parameters: 0.624


In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_train_scaled, y_train)

y_pred = rf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Random Forest Classification Accuracy:", accuracy)

Random Forest Classification Accuracy: 0.575


In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the parameter grid
param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create a Random Forest Classifier
rf = RandomForestClassifier(random_state=42)

# Perform Grid Search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Test Set Accuracy with Best Parameters:", accuracy)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.0s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50; total time=   4.3s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.1s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.2s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   8.3s
[CV] END max_de

In [8]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(random_state=42, max_iter=1000)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Logistic Regression Accuracy:", accuracy)

Logistic Regression Accuracy: 0.513


In [9]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=42)
model.fit(X_train_scaled, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Decision Tree Accuracy:", accuracy)

Decision Tree Accuracy: 0.491


In [19]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score

# Define the parameter grid
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],        # Regularization strength
    'penalty': ['l2'],  # Penalty types
    'solver': ['lbfgs', 'saga'],       # Solvers
    'max_iter': [10000]                    # Ensure convergence
}

# Create the Logistic Regression model
log_reg = LogisticRegression(random_state=42)

# Perform Grid Search
grid_search = GridSearchCV(estimator=log_reg, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validation Accuracy:", grid_search.best_score_)

# Evaluate the best model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
print("Test Set Accuracy with Best Parameters:", accuracy)

Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END ...C=0.01, max_iter=10000, penalty=l2, solver=lbfgs; total time=   1.3s
[CV] END ...C=0.01, max_iter=10000, penalty=l2, solver=lbfgs; total time=   1.4s
[CV] END ...C=0.01, max_iter=10000, penalty=l2, solver=lbfgs; total time=   1.4s
[CV] END ...C=0.01, max_iter=10000, penalty=l2, solver=lbfgs; total time=   1.4s
[CV] END ...C=0.01, max_iter=10000, penalty=l2, solver=lbfgs; total time=   1.4s
[CV] END ....C=0.1, max_iter=10000, penalty=l2, solver=lbfgs; total time=   2.6s
[CV] END ....C=0.1, max_iter=10000, penalty=l2, solver=lbfgs; total time=   2.4s
[CV] END ....C=0.1, max_iter=10000, penalty=l2, solver=lbfgs; total time=   2.4s
[CV] END ....C=0.1, max_iter=10000, penalty=l2, solver=lbfgs; total time=   2.5s
[CV] END ....C=0.1, max_iter=10000, penalty=l2, solver=lbfgs; total time=   2.5s
[CV] END ......C=1, max_iter=10000, penalty=l2, solver=lbfgs; total time=   2.2s
[CV] END ......C=1, max_iter=10000, penalty=l2, 

In [11]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_neighbors=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("KNN Accuracy:", accuracy)

KNN Accuracy: 0.563
